In [1]:
from utils import *
import re
from pprint import pprint
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

numRemover = NumRemover()

all_data3 = get_data_from_db()
all_data3 = numRemover.fit_transform(all_data3)
all_data2 = all_data3[(all_data3["labels"]== "PO") | (all_data3["labels"]== "NG")]

unigramVectorizer = CountVectorizer(ngram_range=(1,1), stop_words= stopwords.words('portuguese'))
bigramVectorizer = CountVectorizer(ngram_range=(2,2), stop_words= stopwords.words('portuguese'))
unibigramVectorizer = CountVectorizer(ngram_range=(1,2), stop_words= stopwords.words('portuguese'))

print("Dimensionalidade")
print("Unigram, 2 labels: ", unigramVectorizer.fit_transform(all_data2['texts']).shape[1])
print("Unigram, 3 labels: ", unigramVectorizer.fit_transform(all_data3['texts']).shape[1])
print("Bigram, 2 labels: ", bigramVectorizer.fit_transform(all_data2['texts']).shape[1])
print("Bigram, 3 labels: ", bigramVectorizer.fit_transform(all_data3['texts']).shape[1])
print("Unibigram, 2 labels: ", unibigramVectorizer.fit_transform(all_data2['texts']).shape[1])
print("Unibigram, 3 labels: ", unibigramVectorizer.fit_transform(all_data3['texts']).shape[1])

def evaluate(data, vectorizer, n_folds):
    #print("Naive Bayes---------------------------------")
    #run_cross_validation(data, vectorizer, MultinomialNB(), n_folds = n_folds)
    print("\nMaxEnt--------------------------------------")
    run_cross_validation(data, vectorizer, LogisticRegressionCV(fit_intercept=False, penalty= 'l2', dual= False), n_folds = n_folds)
    print("\nSVM-----------------------------------------")
    run_cross_validation(data, vectorizer, SVC(C=316), n_folds = n_folds)

Dimensionalidade
Unigram, 2 labels:  4977
Unigram, 3 labels:  5938
Bigram, 2 labels:  15525
Bigram, 3 labels:  21065
Unibigram, 2 labels:  20502
Unibigram, 3 labels:  27003


In [2]:
# Learning algorithms
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegressionCV
from sklearn.svm import SVC

print("********************************************")
print("Unigrams, Positive and Negative Texts")
print("********************************************")
evaluate(all_data2, unigramVectorizer, 10)
print("********************************************")

print("********************************************")
print("Unigrams, Positive, Negative and Neutral Texts")
print("********************************************")
evaluate(all_data3, unigramVectorizer, 10)
print("********************************************")

********************************************
Unigrams, Positive and Negative Texts
********************************************
Naive Bayes---------------------------------
Cross Validation:
Accuracia media:  0.740402414487
Desvio padrão:  0.0590152747254

MaxEnt--------------------------------------
Cross Validation:
Accuracia media:  0.714728370221
Desvio padrão:  0.0520688270576

SVM-----------------------------------------
Cross Validation:
Accuracia media:  0.689054325956
Desvio padrão:  0.041457013093
********************************************
********************************************
Unigrams, Positive, Negative and Neutral Texts
********************************************
Naive Bayes---------------------------------
Cross Validation:
Accuracia media:  0.591074020319
Desvio padrão:  0.0318120872586

MaxEnt--------------------------------------
Cross Validation:
Accuracia media:  0.561284470247
Desvio padrão:  0.037788302971

SVM-----------------------------------------
Cro

In [3]:
print("********************************************")
print("Bigrams, Positive and Negative Texts")
print("********************************************")
evaluate(all_data2, bigramVectorizer, 10)
print("********************************************\n")

print("********************************************")
print("Bigrams, Positive, Negative and Neutral Texts")
print("********************************************")
evaluate(all_data3, bigramVectorizer, 10)
print("********************************************")

********************************************
Bigrams, Positive and Negative Texts
********************************************
Naive Bayes---------------------------------
Cross Validation:
Accuracia media:  0.734486921529
Desvio padrão:  0.0562139478682

MaxEnt--------------------------------------
Cross Validation:
Accuracia media:  0.708913480885
Desvio padrão:  0.054973048298

SVM-----------------------------------------
Cross Validation:
Accuracia media:  0.706116700201
Desvio padrão:  0.0516679829356
********************************************

********************************************
Bigrams, Positive, Negative and Neutral Texts
********************************************
Naive Bayes---------------------------------
Cross Validation:
Accuracia media:  0.554698838897
Desvio padrão:  0.02979894565

MaxEnt--------------------------------------
Cross Validation:
Accuracia media:  0.527703193033
Desvio padrão:  0.0354927271111

SVM-----------------------------------------
Cross

In [4]:
print("********************************************")
print("Unigrams and Bigrams, Positive and Negative Texts")
print("********************************************")
evaluate(all_data2, unibigramVectorizer, 10)
print("********************************************\n")

print("********************************************")
print("Unigrams and Bigrams, Positive, Negative and Neutral Texts")
print("********************************************")
evaluate(all_data3, unibigramVectorizer, 10)
print("********************************************")

********************************************
Unigrams and Bigrams, Positive and Negative Texts
********************************************
Naive Bayes---------------------------------
Cross Validation:
Accuracia media:  0.758913480885
Desvio padrão:  0.0467087610498

MaxEnt--------------------------------------
Cross Validation:
Accuracia media:  0.723319919517
Desvio padrão:  0.0519344270977

SVM-----------------------------------------
Cross Validation:
Accuracia media:  0.70754527163
Desvio padrão:  0.0449155907986
********************************************

********************************************
Unigrams and Bigrams, Positive, Negative and Neutral Texts
********************************************
Naive Bayes---------------------------------
Cross Validation:
Accuracia media:  0.593087808418
Desvio padrão:  0.0333112829391

MaxEnt--------------------------------------
Cross Validation:
Accuracia media:  0.569920174165
Desvio padrão:  0.0315720973231

SVM-------------------

### Usando o PCA com 2 classes

In [27]:
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline, FeatureUnion

pca = PCA()
pca = TruncatedSVD(n_components=701)
cv  = CountVectorizer(stop_words= stopwords.words('portuguese'))

t_data = cv.fit_transform(all_data2["texts"])
t_data = pca.fit_transform(t_data.toarray())

labels = np.array([])
for label in all_data2["labels"]:
    if label == 'PO':
        labels = np.append(labels, [1])
    elif label == "NG":
        labels = np.append(labels, [-1])
    elif label == "NE":
        pass
#         labels.append(0)

labels.shape

(701,)

In [28]:
classifier = LogisticRegressionCV(fit_intercept=False, penalty= 'l2', dual= False)
# classifier = MultinomialNB()

print("Cross Validation:")
accuracy_average = np.array([])
sKFold = StratifiedKFold(n_splits= 10, shuffle= True, random_state= True)
for index, (train, test) in enumerate(sKFold.split(t_data, labels)):
    # Treinando um modelo Naive Bayes
    train_data = t_data[train]
    test_data = t_data[test]

    classifier.fit(train_data, labels[train])
    predictions = classifier.predict(test_data)

    accuracy = accuracy_score(labels[test], predictions, normalize=True)
    # classifier_models.append(pipeline)

    accuracy_average = np.append(accuracy_average, accuracy)
    #print("Fold ", index, " - Acuracia: ", accuracy)

print("Accuracia media: ", accuracy_average.mean())
print("Desvio padrão: ", accuracy_average.std())

Cross Validation:
Accuracia media:  0.711871227364
Desvio padrão:  0.0512239816422


### Usando o pca com 3 classes

In [30]:
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline, FeatureUnion

pca = PCA()
pca = TruncatedSVD(n_components=701)
cv  = CountVectorizer(stop_words= stopwords.words('portuguese'))

t_data = cv.fit_transform(all_data3["texts"])
t_data = pca.fit_transform(t_data.toarray())

labels = np.array([])
for label in all_data3["labels"]:
    if label == 'PO':
        labels = np.append(labels, [1])
    elif label == "NG":
        labels = np.append(labels, [-1])
    elif label == "NE":
        labels = np.append(labels, [0])

labels.shape

classifier = LogisticRegressionCV(fit_intercept=False, penalty= 'l2', dual= False)
# classifier = MultinomialNB()

print("Cross Validation:")
accuracy_average = np.array([])
sKFold = StratifiedKFold(n_splits= 10, shuffle= True, random_state= True)
for index, (train, test) in enumerate(sKFold.split(t_data, labels)):
    # Treinando um modelo Naive Bayes
    train_data = t_data[train]
    test_data = t_data[test]

    classifier.fit(train_data, labels[train])
    predictions = classifier.predict(test_data)

    accuracy = accuracy_score(labels[test], predictions, normalize=True)
    # classifier_models.append(pipeline)

    accuracy_average = np.append(accuracy_average, accuracy)
    #print("Fold ", index, " - Acuracia: ", accuracy)

print("Accuracia media: ", accuracy_average.mean())
print("Desvio padrão: ", accuracy_average.std())

Cross Validation:
Accuracia media:  0.5662191582
Desvio padrão:  0.0447801669792
